## Working with Transformers in the HuggingFace Ecosystem

In this laboratory exercise we will learn how to work with the HuggingFace ecosystem to adapt models to new tasks. As you will see, much of what is required is *investigation* into the inner-workings of the HuggingFace abstractions. With a little work, a little trial-and-error, it is fairly easy to get a working adaptation pipeline up and running.

### Exercise 1: Sentiment Analysis (warm up)

In this first exercise we will start from a pre-trained BERT transformer and build up a model able to perform text sentiment analysis. Transformers are complex beasts, so we will build up our pipeline in several explorative and incremental steps.

#### Exercise 1.1: Dataset Splits and Pre-trained model
There are a many sentiment analysis datasets, but we will use one of the smallest ones available: the [Cornell Rotten Tomatoes movie review dataset](cornell-movie-review-data/rotten_tomatoes), which consists of 5,331 positive and 5,331 negative processed sentences from the Rotten Tomatoes movie reviews.

**Your first task**: Load the dataset and figure out what splits are available and how to get them. Spend some time exploring the dataset to see how it is organized. Note that we will be using the [HuggingFace Datasets](https://huggingface.co/docs/datasets/en/index) library for downloading, accessing, splitting, and batching data for training and evaluation.

In [1]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score
from datasets import load_dataset, get_dataset_split_names
from peft import LoraConfig, get_peft_model

In [2]:
# Your code here.

# Vérifier les splits disponibles
print(get_dataset_split_names("rotten_tomatoes"))

# Charger le dataset complet
dataset = load_dataset("rotten_tomatoes")

# Explorer la structure
print(dataset)
print(dataset["train"][0])  # Premier exemple d'entraînement
print(f"Nombre d'exemples: {len(dataset['train'])}")

Using the latest cached version of the dataset since rotten_tomatoes couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\loric\.cache\huggingface\datasets\rotten_tomatoes\default\0.0.0\aa13bc287fa6fcab6daf52f0dfb9994269ffea28 (last modified on Sat Nov 15 22:33:38 2025).
Using the latest cached version of the dataset since rotten_tomatoes couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\loric\.cache\huggingface\datasets\rotten_tomatoes\default\0.0.0\aa13bc287fa6fcab6daf52f0dfb9994269ffea28 (last modified on Sat Nov 15 22:33:38 2025).


['train', 'validation', 'test']
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})
{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}
Nombre d'exemples: 8530


#### Exercise 1.2: A Pre-trained BERT and Tokenizer

The model we will use is a *very* small BERT transformer called [Distilbert](https://huggingface.co/distilbert/distilbert-base-uncased) this model was trained (using self-supervised learning) on the same corpus as BERT but using the full BERT base model as a *teacher*.

**Your next task**: Load the Distilbert model and corresponding tokenizer. Use the tokenizer on a few samples from the dataset and pass the tokens through the model to see what outputs are provided. I suggest you use the [`AutoModel`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html) class (and the `from_pretrained()` method) to load the model and `AutoTokenizer` to load the tokenizer).

In [3]:
# Your code here.

# Charger le tokenizer et le modèle
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Tester sur un exemple
sample_text = dataset["train"][0]["text"]
inputs = tokenizer(sample_text, return_tensors="pt")
outputs = model(**inputs)

print(f"Shape des sorties: {outputs.last_hidden_state.shape}")
print(f"Keys des outputs: {outputs.keys()}")

Shape des sorties: torch.Size([1, 47, 768])
Keys des outputs: odict_keys(['last_hidden_state'])


#### Exercise 1.3: A Stable Baseline

In this exercise I want you to:
1. Use Distilbert as a *feature extractor* to extract representations of the text strings from the dataset splits;
2. Train a classifier (your choice, by an SVM from Scikit-learn is an easy choice).
3. Evaluate performance on the validation and test splits.

These results are our *stable baseline* -- the **starting** point on which we will (hopefully) improve in the next exercise.

**Hint**: There are a number of ways to implement the feature extractor, but probably the best is to use a [feature extraction `pipeline`](https://huggingface.co/tasks/feature-extraction). You will need to interpret the output of the pipeline and extract only the `[CLS]` token from the *last* transformer layer. *How can you figure out which output that is?*

In [4]:
# Your code here.

# Créer le pipeline d'extraction de features
feature_extractor = pipeline(
    "feature-extraction", 
    model=model_name,
    tokenizer=tokenizer
)

def extract_cls_features(texts):
    """Extraire les features [CLS] de la dernière couche"""
    features = feature_extractor(texts, batch_size=8)
    return np.array([output[0][0] for output in features])  # [CLS] token

# Prendre un échantillon équilibré pour s'assurer d'avoir les deux classes
def get_balanced_sample(texts, labels, sample_size=100):
    """Obtenir un échantillon équilibré des deux classes"""
    positive_indices = [i for i, label in enumerate(labels) if label == 1]
    negative_indices = [i for i, label in enumerate(labels) if label == 0]
    
    # Prendre la moitié de chaque classe
    half_size = sample_size // 2
    selected_indices = (positive_indices[:half_size] + 
                       negative_indices[:half_size])
    
    return [texts[i] for i in selected_indices], [labels[i] for i in selected_indices]

# Obtenir un échantillon équilibré
train_texts, train_labels_sample = get_balanced_sample(
    dataset["train"]["text"], 
    dataset["train"]["label"],
    sample_size=200  # Un peu plus grand pour être sûr
)

# Extraire les features
print("Extraction des features...")
train_features = extract_cls_features(train_texts[:200])  # Utiliser 200 exemples

# Entraîner le classifieur
print("Entraînement du classifieur...")
clf = SVC(kernel='linear', random_state=42)
clf.fit(train_features, train_labels_sample[:200])

# Évaluation sur la validation
print("Évaluation...")
val_texts = dataset["validation"]["text"][:100]  # Prendre 100 exemples pour tester
val_labels = dataset["validation"]["label"][:100]
val_features = extract_cls_features(val_texts)

preds = clf.predict(val_features)
accuracy = accuracy_score(val_labels, preds)

print(f"Accuracy sur validation: {accuracy:.3f}")
print(f"Répartition des prédictions: {np.bincount(preds)}")
print(f"Répartition des vraies labels: {np.bincount(val_labels)}")

# Test sur le split test
test_texts = dataset["test"]["text"][:200]
test_labels = dataset["test"]["label"][:200]
test_features = extract_cls_features(test_texts)

test_preds = clf.predict(test_features)
test_accuracy = accuracy_score(test_labels, test_preds)

print(f"Accuracy sur test: {test_accuracy:.3f}")

Device set to use cuda:0


Extraction des features...


c:\Users\loric\OneDrive\Documents\LM_IA_Unifi\Anaconda3\Lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:401: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Entraînement du classifieur...
Évaluation...
Accuracy sur validation: 0.770
Répartition des prédictions: [23 77]
Répartition des vraies labels: [  0 100]
Accuracy sur test: 0.765


-----
### Exercise 2: Fine-tuning Distilbert

In this exercise we will fine-tune the Distilbert model to (hopefully) improve sentiment analysis performance.

#### Exercise 2.1: Token Preprocessing

The first thing we need to do is *tokenize* our dataset splits. Our current datasets return a dictionary with *strings*, but we want *input token ids* (i.e. the output of the tokenizer). This is easy enough to do my hand, but the HugginFace `Dataset` class provides convenient, efficient, and *lazy* methods. See the documentation for [`Dataset.map`](https://huggingface.co/docs/datasets/v3.5.0/en/package_reference/main_classes#datasets.Dataset.map).

**Tip**: Verify that your new datasets are returning for every element: `text`, `label`, `intput_ids`, and `attention_mask`.

In [5]:
# Your code here.

def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        padding=True, 
        truncation=True,
        max_length=256
    )

# Tokenizer le dataset
tokenized_datasets = dataset.map(
    tokenize_function, 
    batched=True
)

# Nettoyer les colonnes
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

print(tokenized_datasets["train"][0].keys())

dict_keys(['labels', 'input_ids', 'attention_mask'])


#### Exercise 2.2: Setting up the Model to be Fine-tuned

In this exercise we need to prepare the base Distilbert model for fine-tuning for a *sequence classification task*. This means, at the very least, appending a new, randomly-initialized classification head connected to the `[CLS]` token of the last transformer layer. Luckily, HuggingFace already provides an `AutoModel` for just this type of instantiation: [`AutoModelForSequenceClassification`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification). You will want you instantiate one of these for fine-tuning.

In [6]:
# Your code here.

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2  # Classification binaire
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Exercise 2.3: Fine-tuning Distilbert

Finally. In this exercise you should use a HuggingFace [`Trainer`](https://huggingface.co/docs/transformers/main/en/trainer) to fine-tune your model on the Rotten Tomatoes training split. Setting up the trainer will involve (at least):


1. Instantiating a [`DataCollatorWithPadding`](https://huggingface.co/docs/transformers/en/main_classes/data_collator) object which is what *actually* does your batch construction (by padding all sequences to the same length).
2. Writing an *evaluation function* that will measure the classification accuracy. This function takes a single argument which is a tuple containing `(logits, labels)` which you should use to compute classification accuracy (and maybe other metrics like F1 score, precision, recall) and return a `dict` with these metrics.  
3. Instantiating a [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.51.1/en/main_classes/trainer#transformers.TrainingArguments) object using some reasonable defaults.
4. Instantiating a `Trainer` object using your train and validation splits, you data collator, and function to compute performance metrics.
5. Calling `trainer.train()`, waiting, waiting some more, and then calling `trainer.evaluate()` to see how it did.

**Tip**: When prototyping this laboratory I discovered the HuggingFace [Evaluate library](https://huggingface.co/docs/evaluate/en/index) which provides evaluation metrics. However I found it to have insufferable layers of abstraction and getting actual metrics computed. I suggest just using the Scikit-learn metrics...

In [8]:
# Your code here.

# Data collator pour le padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Fonction d'évaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions)
    }

# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Entraînement et évaluation
trainer.train()
results = trainer.evaluate(tokenized_datasets["test"])
print(results)

C:\Users\loric\AppData\Local\Temp\ipykernel_38100\4144902502.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: Currently logged in as: lorictongo (lorictongo-university-of-firenze) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.415600,0.416507,0.818011,0.836700
2,0.254500,0.411208,0.841463,0.845943
3,0.164300,0.493393,0.848968,0.851064


{'eval_loss': 0.4712594747543335, 'eval_accuracy': 0.8395872420262664, 'eval_f1': 0.8449682683590208, 'eval_runtime': 2.9467, 'eval_samples_per_second': 361.759, 'eval_steps_per_second': 22.737, 'epoch': 3.0}


-----
### Exercise 3: Choose at Least One


#### Exercise 3.1: Efficient Fine-tuning for Sentiment Analysis (easy)

In Exercise 2 we fine-tuned the *entire* Distilbert model on Rotten Tomatoes. This is expensive, even for a small model. Find an *efficient* way to fine-tune Distilbert on the Rotten Tomatoes dataset (or some other dataset).

**Hint**: You could check out the [HuggingFace PEFT library](https://huggingface.co/docs/peft/en/index) for some state-of-the-art approaches that should "just work". How else might you go about making fine-tuning more efficient without having to change your training pipeline from above?

In [9]:
# Your code here.


# Fonction d'évaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions)
    }

# Apply LoRA for parameter-efficient fine-tuning
lora_config = LoraConfig(
    r=8,                   # low-rank dimension
    lora_alpha=16,         # scaling factor
    target_modules=["q_lin", "v_lin"],  # which weights to adapt
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, lora_config)

def patch_forward(model):
    original_forward = model.forward
    def new_forward(*args, **kwargs):
        kwargs.pop("num_items_in_batch", None)
        return original_forward(*args, **kwargs)
    model.forward = new_forward

patch_forward(model)

# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    remove_unused_columns=False,
    load_best_model_at_end=False
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Entraînement et évaluation
trainer.train()
results = trainer.evaluate(tokenized_datasets["test"])
print(results)

C:\Users\loric\AppData\Local\Temp\ipykernel_38100\3351198289.py:50: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.142100,0.441362,0.845216,0.846797
2,0.139300,0.434065,0.848968,0.849110
3,0.142800,0.437866,0.849906,0.849906


c:\Users\loric\OneDrive\Documents\LM_IA_Unifi\Anaconda3\Lib\site-packages\peft\utils\other.py:1228: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 77d7c845-b077-4706-bdca-3658f056a0cd)') - silently ignoring the lookup for the file config.json in distilbert-base-uncased.
  warnings.warn(
c:\Users\loric\OneDrive\Documents\LM_IA_Unifi\Anaconda3\Lib\site-packages\peft\utils\save_and_load.py:286: UserWarning: Could not find a config file in distilbert-base-uncased - will assume that the vocabulary was not modified.
  warnings.warn(


{'eval_loss': 0.5005320310592651, 'eval_accuracy': 0.850844277673546, 'eval_f1': 0.8518173345759553, 'eval_runtime': 2.0214, 'eval_samples_per_second': 527.353, 'eval_steps_per_second': 33.145, 'epoch': 3.0}


#### Exercise 3.2: Fine-tuning a CLIP Model (harder)

Use a (small) CLIP model like [`openai/clip-vit-base-patch16`](https://huggingface.co/openai/clip-vit-base-patch16) and evaluate its zero-shot performance on a small image classification dataset like ImageNette or TinyImageNet. Fine-tune (using a parameter-efficient method!) the CLIP model to see how much improvement you can squeeze out of it.

**Note**: There are several ways to adapt the CLIP model; you could fine-tune the image encoder, the text encoder, or both. Or, you could experiment with prompt learning.

**Tip**: CLIP probably already works very well on ImageNet and ImageNet-like images. For extra fun, look for an image classification dataset with different image types (e.g. *sketches*).

In [4]:
# Your code here.

#### Exercise 3.3: Choose your Own Adventure

There are a *ton* of interesting and fun models on the HuggingFace hub. Pick one that does something interesting and adapt it in some way to a new task. Or, combine two or more models into something more interesting or fun. The sky's the limit.

**Note**: Reach out to me by email or on the Discord if you are unsure about anything.

In [5]:
# Your code here.